In [3]:
str = "SHIPMENT_GID,STATUS_TYPE_GID,STATUS_VALUE_GID,DOMAIN_NAME,INSERT_USER,INSERT_DATE,UPDATE_USER,UPDATE_DATE,JOB_NAME,LOAD_DATE"
print(str.lower())

shipment_gid,status_type_gid,status_value_gid,domain_name,insert_user,insert_date,update_user,update_date,job_name,load_date


In [1]:
str="""ORIGIN_CTE AS (
SELECT *
FROM (
SELECT DISTINCT VBAK.VBELN,
VBAP.WERKS as SAP_ORIGIN_ID,
ROW_NUMBER() OVER(PARTITION BY VBAK.VBELN ORDER BY VBAP.ERDAT DESC) AS RN
FROM snowflake_prd_landzone.RAW_SAP.TBL_SAP_VBAK AS VBAK
LEFT JOIN snowflake_prd_landzone.RAW_SAP.TBL_SAP_VBAP AS VBAP
ON VBAK.VBELN = VBAP.VBELN
WHERE VBAP.WERKS <> 'CPCA' AND VBAP.WERKS <> 'CORP'

UNION

SELECT DISTINCT VBAK.VBELN,
VBAP.WERKS as SAP_ORIGIN_ID,
ROW_NUMBER() OVER(PARTITION BY VBAK.VBELN ORDER BY VBAP.ERDAT DESC) AS RN
FROM snowflake_prd_landzone.RAW_SAP.TBL_S4_VBAK AS VBAK
LEFT JOIN snowflake_prd_landzone.RAW_SAP.TBL_S4_VBAP AS VBAP
ON VBAK.VBELN = VBAP.VBELN
WHERE VBAP.WERKS <> 'CPCA' AND VBAP.WERKS <> 'CORP'
)
WHERE RN = 1)"""

print(str.lower())

origin_cte as (
select *
from (
select distinct vbak.vbeln,
vbap.werks as sap_origin_id,
row_number() over(partition by vbak.vbeln order by vbap.erdat desc) as rn
from snowflake_prd_landzone.raw_sap.tbl_sap_vbak as vbak
left join snowflake_prd_landzone.raw_sap.tbl_sap_vbap as vbap
on vbak.vbeln = vbap.vbeln
where vbap.werks <> 'cpca' and vbap.werks <> 'corp'

union

select distinct vbak.vbeln,
vbap.werks as sap_origin_id,
row_number() over(partition by vbak.vbeln order by vbap.erdat desc) as rn
from snowflake_prd_landzone.raw_sap.tbl_s4_vbak as vbak
left join snowflake_prd_landzone.raw_sap.tbl_s4_vbap as vbap
on vbak.vbeln = vbap.vbeln
where vbap.werks <> 'cpca' and vbap.werks <> 'corp'
)
where rn = 1)


In [2]:
str = "otm_ie_shipmentstatus"
print(str.upper())

OTM_IE_SHIPMENTSTATUS


In [1]:
str="""SELECT VBAK.ORDER_NO
FROM ALL_ORDERS_CTE AS VBAK
LEFT JOIN SAP_PGI_CTE AS PGI
ON VBAK.ORDER_NO = PGI.ORDER_NO
LEFT JOIN
(
    SELECT *
    FROM
    (   SELECT OC.*,
        O.COUNT_ORD
        FROM ORD_CODE_CTE as OC
        LEFT JOIN ORD_CTE as O
        ON OC.VBELN = O.VBELN
        WHERE ABGRU = '00'
    )
    WHERE COUNT_ORD_CODE = COUNT_ORD
) AS VBAP
ON VBAK.ORDER_NO = vbap.VBELN
WHERE ORDER_TYPE LIKE ANY ('KL','TA','ZCON','ZDIS')
AND REQUEST_DELIVERY_DATE > DATEADD(YEAR, -1,CURRENT_DATE())
AND (SAP_PGI = '1900-01-01' or SAP_PGI IS NULL)
AND VBAP.VBELN IS NULL
AND VBAK.ORDER_NO NOT IN (
SELECT DISTINCT VBELV
FROM snowflake_prd_landzone.RAW_SAP.TBL_SAP_VBFA
WHERE VBTYP_N = 'R'

UNION

SELECT DISTINCT VBELV
FROM snowflake_prd_landzone.RAW_SAP.TBL_S4_VBFA
WHERE VBTYP_N = 'R'
)
"""
print(str.lower())

select vbak.order_no
from all_orders_cte as vbak
left join sap_pgi_cte as pgi
on vbak.order_no = pgi.order_no
left join
(
    select *
    from
    (   select oc.*,
        o.count_ord
        from ord_code_cte as oc
        left join ord_cte as o
        on oc.vbeln = o.vbeln
        where abgru = '00'
    )
    where count_ord_code = count_ord
) as vbap
on vbak.order_no = vbap.vbeln
where order_type like any ('kl','ta','zcon','zdis')
and request_delivery_date > dateadd(year, -1,current_date())
and (sap_pgi = '1900-01-01' or sap_pgi is null)
and vbap.vbeln is null
and vbak.order_no not in (
select distinct vbelv
from snowflake_prd_landzone.raw_sap.tbl_sap_vbfa
where vbtyp_n = 'r'

union

select distinct vbelv
from snowflake_prd_landzone.raw_sap.tbl_s4_vbfa
where vbtyp_n = 'r'
)



In [1]:
str = """

SELECT ORDER_NO, SUM(QUANTITY_CSU) AS CSU_QUANTITY
FROM
(   SELECT VBAP.VBELN AS ORDER_NO,
    VBAP.MATNR,
    VBAP.KWMENG AS QUANTITY,
    (VBAP.KWMENG * (VBAP.UMZIZ/VBAP.UMZIN)) AS QUANTITY_BUOM,
    ((VBAP.KWMENG * (VBAP.UMZIZ/VBAP.UMZIN)) * (MARM.UMREN/MARM.UMREZ)) AS QUANTITY_CSU,
    VBAP.ZIEME AS UOM,
    VBAP.UMZIZ AS NUMERATOR,
    VBAP.UMZIN AS DENOMINATOR,
    VBAP.MEINS AS BUOM,
    MARM.MEINH AS ALT_UOM,
    MARM.UMREZ AS ALT_NUM,
    MARM.UMREN AS ALT_DENOM
    FROM VBAP
    LEFT JOIN snowflake_prd_landzone.RAW_SAP.TBL_SAP_MARM AS MARM
    ON VBAP.MATNR = MARM.MATNR
    WHERE MARM.MEINH = 'CSU'
)

GROUP BY ORDER_NO
"""

In [3]:
print(str.lower())



select order_no, sum(quantity_csu) as csu_quantity
from
(   select vbap.vbeln as order_no,
    vbap.matnr,
    vbap.kwmeng as quantity,
    (vbap.kwmeng * (vbap.umziz/vbap.umzin)) as quantity_buom,
    ((vbap.kwmeng * (vbap.umziz/vbap.umzin)) * (marm.umren/marm.umrez)) as quantity_csu,
    vbap.zieme as uom,
    vbap.umziz as numerator,
    vbap.umzin as denominator,
    vbap.meins as buom,
    marm.meinh as alt_uom,
    marm.umrez as alt_num,
    marm.umren as alt_denom
    from vbap
    left join snowflake_prd_landzone.raw_sap.tbl_sap_marm as marm
    on vbap.matnr = marm.matnr
    where marm.meinh = 'csu'
)

group by order_no



In [1]:
str = """col_name,data_type,comment
I_TRANSACTION_NO,"decimal(10,0)",""
SERVPROV_ALIAS_QUAL_GID,string,""
SERVPROV_ALIAS_VALUE,string,""
DESCRIPTION,string,""
STATUS_LEVEL,string,""
VESSEL_CODE_QUALIFIER,string,""
VESSEL_CODE,string,""
COUNTRY_CODE3_GID,string,""
VESSEL_GID,string,""
FIGHT_OR_VOYAGE_NUM,string,""
WEIGHT_VALUE_UOM_CODE,string,""
VOLUME_VALUE_UOM_CODE,string,""
OB_REFNUM_QUAL_GID,string,""
OB_REFNUM_VALUE,string,""
STATUS_CODE_GID,string,""
TIME_ZONE_GID,string,""
STATUS_REASON_CODE_GID,string,""
SS_CONTACT_FUNCTION_CODE,string,""
SS_CONTACT_NAME,string,""
TRAIN_JCT_BO_CODE,string,""
RESPONSIBLE_PARTY_GID,string,""
RAIL_ROAD_REPORTING_SCAC,string,""
IS_CAR_LOADED,string,""
CAR_DESTINATION_CITY,string,""
CAR_DESTINATION_STATE,string,""
CAR_DESTINATION_SPLC_CODE,string,""
ORIG_TRANSACTION_NO,"decimal(10,0)",""
TEMPERATURE_UOM_CODE,string,""
DRIVER_GID,string,""
POWER_UNIT_GID,string,""
SHIPMENT_GID,string,""
DRIVER_REFNUM_QUAL_GID,string,""
DRIVER_REFNUM_VALUE,string,""
PU_REFNUM_QUAL_GID,string,""
PU_REFNUM_VALUE,string,""
EXTERNAL_EVENT_REFERENCE,string,""
EXTERNAL_DATA_SOURCE,string,""
TIME_WORKED_UOM_CODE,string,""
TIME_DRIVEN_UOM_CODE,string,""
DRIVER_CAL_EVENT_GID,string,""
MARKED_FOR_PURGE,string,""
DOMAIN_NAME,string,""
INSERT_USER,string,""
UPDATE_USER,string,""
ORDER_BASE_GID,string,""
ORDER_RELEASE_GID,string,""
OR_REFNUM_QUAL_GID,string,""
OR_REFNUM_VALUE,string,""
TRACKING_NUMBER,string,""
CAR_DESTINATION_LOCATION_GID,string,""
CAR_DESTINATION_COUNTRY,string,""
ERPC,string,""
FILING_RESPONSE_TYPE,string,""
ATTRIBUTE1,string,""
ATTRIBUTE2,string,""
ATTRIBUTE3,string,""
ATTRIBUTE4,string,""
ATTRIBUTE5,string,""
ATTRIBUTE6,string,""
ATTRIBUTE7,string,""
ATTRIBUTE8,string,""
ATTRIBUTE9,string,""
ATTRIBUTE10,string,""
ATTRIBUTE11,string,""
ATTRIBUTE12,string,""
ATTRIBUTE13,string,""
ATTRIBUTE14,string,""
ATTRIBUTE15,string,""
ATTRIBUTE16,string,""
ATTRIBUTE17,string,""
ATTRIBUTE18,string,""
ATTRIBUTE19,string,""
ATTRIBUTE20,string,""
SHIP_GROUP_GID,string,""
SHIP_GROUP_TYPE_GID,string,""
ORDER_RELEASE_LINE_GID,string,""
ORL_REFNUM_VALUE,string,""
OR_LINE_REFNUM_QUAL_GID,string,""
INDICATOR,string,""
NEXT_JUNCTION_SPLC,string,""
LOAD_DATE,timestamp,""
JOB_NAME,string,""
AHEAD_LATE_TIME_UOM_CODE,string,""
AHEAD_OR_LATE,string,""
CAR_DEST_TIME_ZONE_GID,string,""
CAR_INTERCHANGE_TIME_ZONE_GID,string,""
EQUIPMENT_INITIAL,string,""
EQUIPMENT_NUMBER,string,""
LOCATION_GID,string,""
SHIPMENT_STATUS_TYPE,string,""
SHIP_OR_SAIL_DATE,timestamp,""
DISCHARGE_OR_COMPLETION_DATE,timestamp,""
EVENTDATE,timestamp,""
CAR_DESTINATION_ETA_DATE,timestamp,""
EVENT_RECEIVED_DATE,timestamp,""
EVENT_END_DATE,timestamp,""
INSERT_DATE,timestamp,""
UPDATE_DATE,timestamp,""
ATTRIBUTE_DATE1,timestamp,""
ATTRIBUTE_DATE2,timestamp,""
ATTRIBUTE_DATE3,timestamp,""
ATTRIBUTE_DATE4,timestamp,""
ATTRIBUTE_DATE5,timestamp,""
ATTRIBUTE_DATE6,timestamp,""
ATTRIBUTE_DATE7,timestamp,""
ATTRIBUTE_DATE8,timestamp,""
ATTRIBUTE_DATE9,timestamp,""
ATTRIBUTE_DATE10,timestamp,""
CAR_INTERCHANGE_ETI_DATE,timestamp,""
WEIGHT_VALUE,"decimal(35,15)",""
WEIGHT_VALUE_BASE,"decimal(35,15)",""
VOLUME_VALUE,"decimal(35,15)",""
VOLUME_VALUE_BASE,"decimal(35,15)",""
PARTITION_KEY,"decimal(35,15)",""
TEMPERATURE,"decimal(35,15)",""
TEMPERATURE_BASE,"decimal(35,15)",""
RELATED_TRANSACTION_NO,"decimal(35,15)",""
TIME_WORKED,"decimal(35,15)",""
TIME_WORKED_BASE,"decimal(35,15)",""
TIME_DRIVEN,"decimal(35,15)",""
TIME_DRIVEN_BASE,"decimal(35,15)",""
ATTRIBUTE_NUMBER1,"decimal(35,15)",""
ATTRIBUTE_NUMBER2,"decimal(35,15)",""
ATTRIBUTE_NUMBER3,"decimal(35,15)",""
ATTRIBUTE_NUMBER4,"decimal(35,15)",""
ATTRIBUTE_NUMBER5,"decimal(35,15)",""
ATTRIBUTE_NUMBER6,"decimal(35,15)",""
ATTRIBUTE_NUMBER7,"decimal(35,15)",""
ATTRIBUTE_NUMBER8,"decimal(35,15)",""
ATTRIBUTE_NUMBER9,"decimal(35,15)",""
ATTRIBUTE_NUMBER10,"decimal(35,15)",""
CAR_REPAIR_HOURS,"decimal(35,15)",""
AHEAD_LATE_TIME,"decimal(35,15)",""
AHEAD_LATE_TIME_BASE,"decimal(35,15)",""
"""
print(str.lower())

col_name,data_type,comment
i_transaction_no,"decimal(10,0)",""
servprov_alias_qual_gid,string,""
servprov_alias_value,string,""
description,string,""
status_level,string,""
vessel_code_qualifier,string,""
vessel_code,string,""
country_code3_gid,string,""
vessel_gid,string,""
fight_or_voyage_num,string,""
weight_value_uom_code,string,""
volume_value_uom_code,string,""
ob_refnum_qual_gid,string,""
ob_refnum_value,string,""
status_code_gid,string,""
time_zone_gid,string,""
status_reason_code_gid,string,""
ss_contact_function_code,string,""
ss_contact_name,string,""
train_jct_bo_code,string,""
responsible_party_gid,string,""
rail_road_reporting_scac,string,""
is_car_loaded,string,""
car_destination_city,string,""
car_destination_state,string,""
car_destination_splc_code,string,""
orig_transaction_no,"decimal(10,0)",""
temperature_uom_code,string,""
driver_gid,string,""
power_unit_gid,string,""
shipment_gid,string,""
driver_refnum_qual_gid,string,""
driver_refnum_value,string,""
pu_refnum_q

In [2]:
str = """I_TRANSACTION_NO,SERVPROV_ALIAS_QUAL_GID,SERVPROV_ALIAS_VALUE,DESCRIPTION,STATUS_LEVEL,VESSEL_CODE_QUALIFIER,VESSEL_CODE,COUNTRY_CODE3_GID,VESSEL_GID,FIGHT_OR_VOYAGE_NUM,WEIGHT_VALUE_UOM_CODE,VOLUME_VALUE_UOM_CODE,OB_REFNUM_QUAL_GID,OB_REFNUM_VALUE,STATUS_CODE_GID,TIME_ZONE_GID,STATUS_REASON_CODE_GID,SS_CONTACT_FUNCTION_CODE,SS_CONTACT_NAME,TRAIN_JCT_BO_CODE,RESPONSIBLE_PARTY_GID,RAIL_ROAD_REPORTING_SCAC,IS_CAR_LOADED,CAR_DESTINATION_CITY,CAR_DESTINATION_STATE,CAR_DESTINATION_SPLC_CODE,ORIG_TRANSACTION_NO,TEMPERATURE_UOM_CODE,DRIVER_GID,POWER_UNIT_GID,SHIPMENT_GID,DRIVER_REFNUM_QUAL_GID,DRIVER_REFNUM_VALUE,PU_REFNUM_QUAL_GID,PU_REFNUM_VALUE,EXTERNAL_EVENT_REFERENCE,EXTERNAL_DATA_SOURCE,TIME_WORKED_UOM_CODE,TIME_DRIVEN_UOM_CODE,DRIVER_CAL_EVENT_GID,MARKED_FOR_PURGE,DOMAIN_NAME,INSERT_USER,UPDATE_USER,ORDER_BASE_GID,ORDER_RELEASE_GID,OR_REFNUM_QUAL_GID,OR_REFNUM_VALUE,TRACKING_NUMBER,CAR_DESTINATION_LOCATION_GID,CAR_DESTINATION_COUNTRY,ERPC,FILING_RESPONSE_TYPE,ATTRIBUTE1,ATTRIBUTE2,ATTRIBUTE3,ATTRIBUTE4,ATTRIBUTE5,ATTRIBUTE6,ATTRIBUTE7,ATTRIBUTE8,ATTRIBUTE9,ATTRIBUTE10,ATTRIBUTE11,ATTRIBUTE12,ATTRIBUTE13,ATTRIBUTE14,ATTRIBUTE15,ATTRIBUTE16,ATTRIBUTE17,ATTRIBUTE18,ATTRIBUTE19,ATTRIBUTE20,SHIP_GROUP_GID,SHIP_GROUP_TYPE_GID,ORDER_RELEASE_LINE_GID,ORL_REFNUM_VALUE,OR_LINE_REFNUM_QUAL_GID,INDICATOR,NEXT_JUNCTION_SPLC,LOAD_DATE,JOB_NAME,AHEAD_LATE_TIME_UOM_CODE,AHEAD_OR_LATE,CAR_DEST_TIME_ZONE_GID,CAR_INTERCHANGE_TIME_ZONE_GID,EQUIPMENT_INITIAL,EQUIPMENT_NUMBER,LOCATION_GID,SHIPMENT_STATUS_TYPE,SHIP_OR_SAIL_DATE,DISCHARGE_OR_COMPLETION_DATE,EVENTDATE,CAR_DESTINATION_ETA_DATE,EVENT_RECEIVED_DATE,EVENT_END_DATE,INSERT_DATE,UPDATE_DATE,ATTRIBUTE_DATE1,ATTRIBUTE_DATE2,ATTRIBUTE_DATE3,ATTRIBUTE_DATE4,ATTRIBUTE_DATE5,ATTRIBUTE_DATE6,ATTRIBUTE_DATE7,ATTRIBUTE_DATE8,ATTRIBUTE_DATE9,ATTRIBUTE_DATE10,CAR_INTERCHANGE_ETI_DATE,WEIGHT_VALUE,WEIGHT_VALUE_BASE,VOLUME_VALUE,VOLUME_VALUE_BASE,PARTITION_KEY,TEMPERATURE,TEMPERATURE_BASE,RELATED_TRANSACTION_NO,TIME_WORKED,TIME_WORKED_BASE,TIME_DRIVEN,TIME_DRIVEN_BASE,ATTRIBUTE_NUMBER1,ATTRIBUTE_NUMBER2,ATTRIBUTE_NUMBER3,ATTRIBUTE_NUMBER4,ATTRIBUTE_NUMBER5,ATTRIBUTE_NUMBER6,ATTRIBUTE_NUMBER7,ATTRIBUTE_NUMBER8,ATTRIBUTE_NUMBER9,ATTRIBUTE_NUMBER10,CAR_REPAIR_HOURS,AHEAD_LATE_TIME,AHEAD_LATE_TIME_BASE"""
print(str.lower())

i_transaction_no,servprov_alias_qual_gid,servprov_alias_value,description,status_level,vessel_code_qualifier,vessel_code,country_code3_gid,vessel_gid,fight_or_voyage_num,weight_value_uom_code,volume_value_uom_code,ob_refnum_qual_gid,ob_refnum_value,status_code_gid,time_zone_gid,status_reason_code_gid,ss_contact_function_code,ss_contact_name,train_jct_bo_code,responsible_party_gid,rail_road_reporting_scac,is_car_loaded,car_destination_city,car_destination_state,car_destination_splc_code,orig_transaction_no,temperature_uom_code,driver_gid,power_unit_gid,shipment_gid,driver_refnum_qual_gid,driver_refnum_value,pu_refnum_qual_gid,pu_refnum_value,external_event_reference,external_data_source,time_worked_uom_code,time_driven_uom_code,driver_cal_event_gid,marked_for_purge,domain_name,insert_user,update_user,order_base_gid,order_release_gid,or_refnum_qual_gid,or_refnum_value,tracking_number,car_destination_location_gid,car_destination_country,erpc,filing_response_type,attribute1,attribute2,attr

In [1]:
str = """SHIPMENT_GID,SHIPMENT_REFNUM_QUAL_GID,SHIPMENT_REFNUM_VALUE,DOMAIN_NAME,INSERT_USER,INSERT_DATE,UPDATE_USER,UPDATE_DATE,JOB_NAME,LOAD_DATE"""
print(str.lower())

shipment_gid,shipment_refnum_qual_gid,shipment_refnum_value,domain_name,insert_user,insert_date,update_user,update_date,job_name,load_date


In [1]:
str = """ORDER_RELEASE_GID,ORDER_RELEASE_XID,ORDER_RELEASE_NAME,ORDER_RELEASE_TYPE_GID,IS_TEMPLATE,ORDER_BASE_GID,SOURCE_LOCATION_GID,DEST_LOCATION_GID,EARLY_PICKUP_DATE,LATE_PICKUP_DATE,EARLY_DELIVERY_DATE,LATE_DELIVERY_DATE,FIXED_ITINERARY_GID,ASSIGNED_ITINERARY_GID,FIXED_SELL_ITINERARY_GID,SHIP_UNIT_ORIGIN,TOTAL_WEIGHT,TOTAL_WEIGHT_UOM_CODE,TOTAL_WEIGHT_BASE,TOTAL_VOLUME,TOTAL_VOLUME_UOM_CODE,TOTAL_VOLUME_BASE,TOTAL_PACKAGING_UNIT_COUNT,TOTAL_ITEM_PACKAGE_COUNT,MAX_LENGTH,MAX_LENGTH_UOM_CODE,MAX_LENGTH_BASE,MAX_WIDTH,MAX_WIDTH_UOM_CODE,MAX_WIDTH_BASE,MAX_HEIGHT,MAX_HEIGHT_UOM_CODE,MAX_HEIGHT_BASE,PLAN_FROM_LOCATION_GID,PLAN_TO_LOCATION_GID,ASSIGNED_SELL_ITINERARY_GID,INDICATOR,BULK_PLAN_GID,BEST_DIRECT_COST_BUY,BEST_DIRCOST_BUY_CURRENCY_GID,BEST_DIRECT_COST_BUY_BASE,BEST_DIRECT_COST_SELL,BEST_DIRCOST_SELL_CURRENCY_GID,BEST_DIRECT_COST_SELL_BASE,BEST_DIRECT_RATEOFFER_GID_BUY,BEST_DIRECT_RATEOFFER_GID_SELL,BUY_GENERAL_LEDGER_GID,SELL_GENERAL_LEDGER_GID,TOTAL_NET_WEIGHT,TOTAL_NET_WEIGHT_UOM_CODE,TOTAL_NET_WEIGHT_BASE,TOTAL_NET_VOLUME,TOTAL_NET_VOLUME_UOM_CODE,TOTAL_NET_VOLUME_BASE,SELL_BULK_PLAN_GID,TOTAL_SHIP_UNIT_COUNT,LOAD_POINT,UNLOAD_POINT,PICKUP_IS_APPT,DELIVERY_IS_APPT,PLAN_FROM_LOAD_POINT,PLAN_TO_UNLOAD_POINT,USER_DEFINED1_ICON_GID,USER_DEFINED2_ICON_GID,USER_DEFINED3_ICON_GID,USER_DEFINED4_ICON_GID,USER_DEFINED5_ICON_GID,EXTERNAL_SYSTEM_ID,PAYMENT_METHOD_CODE_GID,INCO_TERM_GID,TERM_LOCATION_TEXT,PLANNING_GROUP_GID,TIME_WINDOW_EMPHASIS_GID,IS_KNOWN_SHIPPER,DIM_RATE_FACTOR_GID,RELEASE_METHOD_GID,FINAL_INCO_TERM,FINAL_TEXT_LOCATION,INSPECTION_NUMBER,INSPECTION_SCHEDULE_DATE,INSPECTION_DATE,SURVEY_LAY_CAN_DATE,LOC_NUMBER,LOC_ISSUE_DATE,LOC_RECEIVED_DATE,LOC_LATEST_SHIPPING_DATE,LOC_NEGOTIATED_DAYS,LOC_AMOUNT,LOC_AMOUNT_CURRENCY_GID,LOC_AMOUNT_BASE,LOC_ADVISING_NUMBER,LOC_CONFIRMING_NUMBER,SRC_LOCATION_OVERRIDE_GID,DEST_LOCATION_OVERRIDE_GID,PLAN_FROM_LOC_OVERRIDE_GID,PLAN_TO_LOC_OVERRIDE_GID,SECONDARY_T_WEIGHT,SECONDARY_T_WEIGHT_UOM_CODE,SECONDARY_T_WEIGHT_BASE,SECONDARY_T_VOLUME,SECONDARY_T_VOLUME_UOM_CODE,SECONDARY_T_VOLUME_BASE,RATE_SERVICE_PROFILE_GID,SERVPROV_GID,SERVPROV_PROFILE_GID,TRANSPORT_MODE_GID,MODE_PROFILE_GID,EQUIPMENT_GROUP_GID,EQUIPMENT_GROUP_PROFILE_GID,RATE_OFFERING_GID,RATE_GEO_GID,SELL_RATE_OFFERING_GID,SELL_RATE_GEO_GID,MUST_SHIP_DIRECT,MUST_SHIP_THRU_X_DOCK,MUST_SHIP_THRU_POOL,SHIP_WITH_GROUP,PRIMARY_LEG_SOURCE_LOC_GID,PRIM_LEG_SRC_LOC_OVERRIDE_GID,PORT_OF_LOAD_LOCATION_GID,PORT_OF_LOAD_LOC_OVERRIDE_GID,PORT_OF_DIS_LOCATION_GID,PORT_OF_DIS_LOC_OVERRIDE_GID,PRIMARY_LEG_DEST_LOC_GID,PRIM_LEG_DEST_OVERRIDE_LOC_GID,IMPORT_LICENSE_REQUIRED,IMPORT_LICENSE_ISSUE_DATE,IMPORT_LICENSE_RECEIVE_DATE,IMPORT_LICENSE_EXPIRATION_DATE,IMPORT_LICENSE_NUMBER,LOC_EXPIRATION_DATE,IS_LOC_REQUIRED,INSPECTION_REQUIRED,UNITIZATION_CONDITION_NAME,STUFF_LOCATION_GID,DESTUFF_LOCATION_GID,CUSTOMER_UNITIZATION_REQUEST,DEFAULT_STOWAGE_MODE,CONSOLIDATION_TYPE_GID,MOVE_PERSPECTIVE_GID,QUOTE_GID,COMMERCIAL_PAYMENT_CODE_GID,IS_LOC_STALE,ULT_DEST_COUNTRY_CODE3_GID,TEMPLATE_TYPE,BUNDLING_TYPE,IS_CONSOLIDATE_OR_EQUIPMENT,DUTY_PAID,QUOTE_OPTION_NUMBER,PARTITION_GID,TOTAL_DECLARED_VALUE,TOTAL_DECLARED_VALUE_GID,TOTAL_DECLARED_VALUE_BASE,BUY_ITINERARY_PROFILE_GID,SELL_ITINERARY_PROFILE_GID,BUFFER_TYPE,SELL_SERVPROV_GID,SELL_SERVPROV_PROFILE_GID,IS_IGNORE_LOCATION_CALENDAR,RATE_SERVICE_GID,SHIP_UNIT_WIDTH,SHIP_UNIT_WIDTH_UOM_CODE,SHIP_UNIT_WIDTH_BASE,SHIP_UNIT_LENGTH,SHIP_UNIT_LENGTH_UOM_CODE,SHIP_UNIT_LENGTH_BASE,SHIP_UNIT_HEIGHT,SHIP_UNIT_HEIGHT_UOM_CODE,SHIP_UNIT_HEIGHT_BASE,SHIP_UNIT_DIAMETER,SHIP_UNIT_DIAMETER_UOM_CODE,SHIP_UNIT_DIAMETER_BASE,TRANSPORT_HANDLING_UNIT_GID,ON_RT_EXECUTION,IS_SPLITTABLE,PICKUP_ROUTING_SEQUENCE_GID,DROPOFF_ROUTING_SEQUENCE_GID,LATEST_EST_DELIVERY_DATE,OTM_VERSION,PRIORITY,LATEST_EST_PICKUP_DATE,MARKED_FOR_PURGE,PICKUP_RAIL_CARRIER,DELIVERY_RAIL_CARRIER,EQUIPMENT_TYPE_GID,DOMAIN_NAME,INSERT_USER,INSERT_DATE,UPDATE_USER,UPDATE_DATE,RAIL_ROUTE_CODE_GID,EM_PHONE_NUMBER,LEG_CONSOLIDATION_GROUP_GID,IS_PRE_ENTERED_PU,ROUTING_CONSTRAINT_GID,ATTRIBUTE1,ATTRIBUTE2,ATTRIBUTE3,ATTRIBUTE4,ATTRIBUTE5,ATTRIBUTE6,ATTRIBUTE7,ATTRIBUTE8,ATTRIBUTE9,ATTRIBUTE10,ATTRIBUTE11,ATTRIBUTE12,ATTRIBUTE13,ATTRIBUTE14,ATTRIBUTE15,ATTRIBUTE16,ATTRIBUTE17,ATTRIBUTE18,ATTRIBUTE19,ATTRIBUTE20,ATTRIBUTE_NUMBER1,ATTRIBUTE_NUMBER2,ATTRIBUTE_NUMBER3,ATTRIBUTE_NUMBER4,ATTRIBUTE_NUMBER5,ATTRIBUTE_NUMBER6,ATTRIBUTE_NUMBER7,ATTRIBUTE_NUMBER8,ATTRIBUTE_NUMBER9,ATTRIBUTE_NUMBER10,ATTRIBUTE_DATE1,ATTRIBUTE_DATE2,ATTRIBUTE_DATE3,ATTRIBUTE_DATE4,ATTRIBUTE_DATE5,ATTRIBUTE_DATE6,ATTRIBUTE_DATE7,ATTRIBUTE_DATE8,ATTRIBUTE_DATE9,ATTRIBUTE_DATE10,ATTRIBUTE_CURRENCY1,ATTRIBUTE_CURRENCY1_GID,ATTRIBUTE_CURRENCY1_BASE,ATTRIBUTE_CURRENCY2,ATTRIBUTE_CURRENCY2_GID,ATTRIBUTE_CURRENCY2_BASE,ATTRIBUTE_CURRENCY3,ATTRIBUTE_CURRENCY3_GID,ATTRIBUTE_CURRENCY3_BASE,ALLOCATION_GROUP_GID,EARLIEST_EST_PICKUP_DATE,JOB_NAME,LOAD_DATE,ATTRIBUTE_DATE11,ATTRIBUTE_DATE12,ATTRIBUTE_DATE13,ATTRIBUTE_DATE14,ATTRIBUTE_DATE15,ATTRIBUTE_DATE16,ATTRIBUTE_DATE17,ATTRIBUTE_DATE18,ATTRIBUTE_DATE19,ATTRIBUTE_DATE20,ATTRIBUTE_NUMBER11,ATTRIBUTE_NUMBER12,ATTRIBUTE_NUMBER13,ATTRIBUTE_NUMBER14,ATTRIBUTE_NUMBER15,ATTRIBUTE_NUMBER16,ATTRIBUTE_NUMBER17,ATTRIBUTE_NUMBER18,ATTRIBUTE_NUMBER19,ATTRIBUTE_NUMBER20,EARLY_SAIL_DATE,EXTERNAL_VOYAGE_CODE,IS_DIRECT_VOYAGE,IS_TOPOFF,LATE_SAIL_DATE,VOYAGE_GID"""
print(str.lower())

order_release_gid,order_release_xid,order_release_name,order_release_type_gid,is_template,order_base_gid,source_location_gid,dest_location_gid,early_pickup_date,late_pickup_date,early_delivery_date,late_delivery_date,fixed_itinerary_gid,assigned_itinerary_gid,fixed_sell_itinerary_gid,ship_unit_origin,total_weight,total_weight_uom_code,total_weight_base,total_volume,total_volume_uom_code,total_volume_base,total_packaging_unit_count,total_item_package_count,max_length,max_length_uom_code,max_length_base,max_width,max_width_uom_code,max_width_base,max_height,max_height_uom_code,max_height_base,plan_from_location_gid,plan_to_location_gid,assigned_sell_itinerary_gid,indicator,bulk_plan_gid,best_direct_cost_buy,best_dircost_buy_currency_gid,best_direct_cost_buy_base,best_direct_cost_sell,best_dircost_sell_currency_gid,best_direct_cost_sell_base,best_direct_rateoffer_gid_buy,best_direct_rateoffer_gid_sell,buy_general_ledger_gid,sell_general_ledger_gid,total_net_weight,total_net_weight_uom_cod

In [2]:
str = """ORDER_MOVEMENT_GID,ORDER_MOVEMENT_XID,ORDER_RELEASE_GID,IS_TEMPORARY,CREATION_PROCESS_TYPE,SEQ_NO,PERSPECTIVE,SOURCE_LOCATION_GID,IS_FIXED_SRC,DEST_LOCATION_GID,IS_FIXED_DEST,EARLY_PICKUP_DATE,OPERATION_EARLY_PICKUP_DATE,LATE_PICKUP_DATE,EARLY_DELIVERY_DATE,LATE_DELIVERY_DATE,OPERATION_LATE_DELIVERY_DATE,REUSE_EQUIPMENT,PARENT_SOURCE_LOCATION_GID,PARENT_DEST_LOCATION_GID,ORIGINAL_LEG_GID,BULK_PLAN_GID,ORIGINAL_SELL_OM_GID,TOTAL_WEIGHT_UOM_CODE,TOTAL_VOLUME_UOM_CODE,USER_DEFINED1_ICON_GID,USER_DEFINED2_ICON_GID,USER_DEFINED3_ICON_GID,USER_DEFINED4_ICON_GID,USER_DEFINED5_ICON_GID,SHIP_UNIT_WIDTH,SHIP_UNIT_WIDTH_UOM_CODE,SHIP_UNIT_WIDTH_BASE,SHIP_UNIT_LENGTH,SHIP_UNIT_LENGTH_UOM_CODE,SHIP_UNIT_LENGTH_BASE,SHIP_UNIT_HEIGHT,SHIP_UNIT_HEIGHT_UOM_CODE,SHIP_UNIT_HEIGHT_BASE,SHIP_UNIT_DIAMETER,SHIP_UNIT_DIAMETER_UOM_CODE,SHIP_UNIT_DIAMETER_BASE,TRANSPORT_HANDLING_UNIT_GID,INDICATOR,PICKUP_ROUTING_SEQUENCE_GID,DROPOFF_ROUTING_SEQUENCE_GID,PRIORITY,SOURCE_TAG,DEST_TAG,SHIPMENT_GID,DOMAIN_NAME,INSERT_USER,INSERT_DATE,UPDATE_USER,UPDATE_DATE,MODE_PROFILE_GID,EQUIPMENT_GROUP_PROFILE_GID,CALCULATE_SERVICE_TIME,CALCULATE_CONTRACTED_RATE,EXPECTED_TRANSIT_TIME_UOM_CODE,EXPECTED_COST_CURRENCY_GID,RATE_OFFERING_GID,RATE_GEO_GID,EQUIPMENT_GROUP_GID,TRANSPORT_MODE_GID,SERVPROV_GID,SERVPROV_PROFILE_GID,RATE_SERVICE_PROFILE_GID,IS_TEMPLATE,SHIP_WITH_GROUP,LEG_CLASSIFICATION_GID,LEG_CONSOLIDATION_GROUP_GID,IS_NETWORK_ROUTABLE,ROUTING_CONSTRAINT_GID,IS_ROUTING_FIXED,ATTRIBUTE1,ATTRIBUTE2,ATTRIBUTE3,ATTRIBUTE4,ATTRIBUTE5,ATTRIBUTE6,ATTRIBUTE7,ATTRIBUTE8,ATTRIBUTE9,ATTRIBUTE10,ATTRIBUTE11,ATTRIBUTE12,ATTRIBUTE13,ATTRIBUTE14,ATTRIBUTE15,ATTRIBUTE16,ATTRIBUTE17,ATTRIBUTE18,ATTRIBUTE19,ATTRIBUTE20,ATTRIBUTE_NUMBER1,ATTRIBUTE_NUMBER2,ATTRIBUTE_NUMBER3,ATTRIBUTE_NUMBER4,ATTRIBUTE_NUMBER5,ATTRIBUTE_NUMBER6,ATTRIBUTE_NUMBER7,ATTRIBUTE_NUMBER8,ATTRIBUTE_NUMBER9,ATTRIBUTE_NUMBER10,ATTRIBUTE_DATE1,ATTRIBUTE_DATE2,ATTRIBUTE_DATE3,ATTRIBUTE_DATE4,ATTRIBUTE_DATE5,ATTRIBUTE_DATE6,ATTRIBUTE_DATE7,ATTRIBUTE_DATE8,ATTRIBUTE_DATE9,ATTRIBUTE_DATE10,LOADING_SPLIT_TYPE,ORIGINAL_BUY_OM_GID,LOAD_DATE,JOB_NAME,ATTRIBUTE_DATE11,ATTRIBUTE_DATE12,ATTRIBUTE_DATE13,ATTRIBUTE_DATE14,ATTRIBUTE_DATE15,ATTRIBUTE_DATE16,ATTRIBUTE_DATE17,ATTRIBUTE_DATE18,ATTRIBUTE_DATE19,ATTRIBUTE_DATE20,ATTRIBUTE_NUMBER11,ATTRIBUTE_NUMBER12,ATTRIBUTE_NUMBER13,ATTRIBUTE_NUMBER14,ATTRIBUTE_NUMBER15,ATTRIBUTE_NUMBER16,ATTRIBUTE_NUMBER17,ATTRIBUTE_NUMBER18,ATTRIBUTE_NUMBER19,ATTRIBUTE_NUMBER20,ORIGINAL_LEG_POSITION,TOTAL_SHIP_UNIT_COUNT,TOTAL_WEIGHT,TOTAL_WEIGHT_BASE,TOTAL_VOLUME,TOTAL_VOLUME_BASE,EXPECTED_TRANSIT_TIME,EXPECTED_TRANSIT_TIME_BASE,EXPECTED_COST,EXPECTED_COST_BASE"""
print(str.lower())

order_movement_gid,order_movement_xid,order_release_gid,is_temporary,creation_process_type,seq_no,perspective,source_location_gid,is_fixed_src,dest_location_gid,is_fixed_dest,early_pickup_date,operation_early_pickup_date,late_pickup_date,early_delivery_date,late_delivery_date,operation_late_delivery_date,reuse_equipment,parent_source_location_gid,parent_dest_location_gid,original_leg_gid,bulk_plan_gid,original_sell_om_gid,total_weight_uom_code,total_volume_uom_code,user_defined1_icon_gid,user_defined2_icon_gid,user_defined3_icon_gid,user_defined4_icon_gid,user_defined5_icon_gid,ship_unit_width,ship_unit_width_uom_code,ship_unit_width_base,ship_unit_length,ship_unit_length_uom_code,ship_unit_length_base,ship_unit_height,ship_unit_height_uom_code,ship_unit_height_base,ship_unit_diameter,ship_unit_diameter_uom_code,ship_unit_diameter_base,transport_handling_unit_gid,indicator,pickup_routing_sequence_gid,dropoff_routing_sequence_gid,priority,source_tag,dest_tag,shipment_gid,domain_name,ins